In [1]:
from graph_traffic.config import project_path
import pandas as pd
import pickle

training_folder = f"{project_path}/training_history/regression"
training_datetime = "20220530_202406"
#training_datetime = "20220525_083624"

with open(f"{training_folder}/{training_datetime}_meteo_values.pkl", "rb") as f:
    meteo_values = pickle.load(f)

with open(f"{training_folder}/{training_datetime}_temporal_values.pkl", "rb") as f:
    temporal_values = pickle.load(f)

with open(f"{training_folder}/{training_datetime}_results.pkl", "rb") as f:
    results = pickle.load(f)

with open(f"{training_folder}/{training_datetime}_times.pkl", "rb") as f:
    training_time = pickle.load(f)

with open(f"{training_folder}/{training_datetime}_alphas.pkl", "rb") as f:
    alphas = pickle.load(f)

In [2]:
meteo_values

{0: {'rain': 'passthrough',
  'wind': 'drop',
  'temperature': 'passthrough',
  'humidity': 'passthrough',
  'pressure': 'drop',
  'radiation': 'passthrough'},
 1: {'rain': 'passthrough',
  'wind': 'drop',
  'temperature': 'passthrough',
  'humidity': 'passthrough',
  'pressure': 'drop',
  'radiation': 'drop'},
 2: {'rain': 'numerico_power',
  'wind': 'drop',
  'temperature': 'passthrough',
  'humidity': 'passthrough',
  'pressure': 'drop',
  'radiation': 'passthrough'},
 3: {'rain': 'numerico_power',
  'wind': 'drop',
  'temperature': 'passthrough',
  'humidity': 'passthrough',
  'pressure': 'drop',
  'radiation': 'drop'},
 4: {'rain': 'ordinal',
  'wind': 'drop',
  'temperature': 'passthrough',
  'humidity': 'passthrough',
  'pressure': 'drop',
  'radiation': 'passthrough'},
 5: {'rain': 'ordinal',
  'wind': 'drop',
  'temperature': 'passthrough',
  'humidity': 'passthrough',
  'pressure': 'drop',
  'radiation': 'drop'},
 6: {'rain': 'numerico_quantile_normal',
  'wind': 'drop',
  't

In [3]:
temporal_values

{18000: {'year': 'passthrough',
  'season': 'ordinal',
  'month': 'passthrough',
  'day_of_month': 'passthrough',
  'weekday': 'passthrough',
  'hour': 'passthrough',
  'minute': 'passthrough',
  'bank_holiday': 'passthrough',
  'school_holiday': 'passthrough',
  'working_day': 'passthrough',
  'state_of_alarm': 'passthrough'},
 18001: {'year': 'passthrough',
  'season': 'ordinal',
  'month': 'passthrough',
  'day_of_month': 'passthrough',
  'weekday': 'passthrough',
  'hour': 'passthrough',
  'minute': 'passthrough',
  'bank_holiday': 'passthrough',
  'school_holiday': 'passthrough',
  'working_day': 'passthrough',
  'state_of_alarm': 'drop'},
 18002: {'year': 'passthrough',
  'season': 'ordinal',
  'month': 'passthrough',
  'day_of_month': 'passthrough',
  'weekday': 'passthrough',
  'hour': 'passthrough',
  'minute': 'passthrough',
  'bank_holiday': 'passthrough',
  'school_holiday': 'passthrough',
  'working_day': 'drop',
  'state_of_alarm': 'passthrough'},
 18003: {'year': 'passth

In [4]:
def mean_loss(l):
    mae = [x[0] for x in l]
    mse = [x[1] for x in l]
    return sum(mae)/len(mae), sum(mse)/len(mse)

In [20]:
results_summary = {k: mean_loss(l) for (k, l) in results.items()}
df = pd.DataFrame()
df["mae"] = pd.Series({k: v[0] for (k, v) in results_summary.items()})
df["mse"] = pd.Series({k: v[1] for (k, v) in results_summary.items()})
df["mae_rank"] = df["mae"].rank()
df["mse_rank"] = df["mse"].rank()
df["max_rank"] = df[["mae_rank", "mse_rank"]].values.max(1)
df = df.sort_values("max_rank").reset_index()
df.head(20)

,level_0,level_1,mae,mse,mae_rank,mse_rank,max_rank
0,2,18548,2.088017,9.110779,7.0,15.0,15.0
1,0,19988,2.088635,9.100739,16.0,1.0,16.0
2,4,18548,2.087459,9.110986,1.0,16.0,16.0
3,4,18068,2.088553,9.111761,14.0,18.0,18.0
4,6,18548,2.087583,9.112486,2.5,19.5,19.5
5,8,18548,2.087583,9.112486,2.5,19.5,19.5
6,6,18068,2.088703,9.113276,20.5,21.5,21.5
7,8,18068,2.088703,9.113276,20.5,21.5,21.5
8,2,18532,2.088298,9.116596,10.0,31.0,31.0
9,4,18532,2.087742,9.116855,4.0,34.0,34.0


In [21]:
df[df["level_1"].apply(lambda x: temporal_values[x]["hour"]) != "one_hot"].sort_values("max_rank").head(10)

,level_0,level_1,mae,mse,mae_rank,mse_rank,max_rank
320,4,19588,2.141823,9.393259,404.0,389.0,404.0
321,4,19604,2.141904,9.389957,407.0,368.0,407.0
322,6,19588,2.142263,9.395796,423.5,401.5,423.5
323,8,19588,2.142263,9.395796,423.5,401.5,423.5
324,4,18628,2.142318,9.381730,425.0,339.0,425.0
325,8,19604,2.142337,9.392515,427.5,385.5,427.5
326,6,19604,2.142337,9.392515,427.5,385.5,427.5
327,2,19588,2.142398,9.393755,431.0,391.0,431.0
328,2,19604,2.142469,9.390475,432.0,372.0,432.0
329,6,18628,2.142493,9.383202,433.5,343.5,433.5


In [22]:
meteo_values[4]

{'rain': 'ordinal',
 'wind': 'drop',
 'temperature': 'passthrough',
 'humidity': 'passthrough',
 'pressure': 'drop',
 'radiation': 'passthrough'}

In [23]:
temporal_values[19588]

{'year': 'passthrough',
 'season': 'ordinal',
 'month': 'passthrough',
 'day_of_month': 'fourier',
 'weekday': 'passthrough',
 'hour': 'spline',
 'minute': 'passthrough',
 'bank_holiday': 'passthrough',
 'school_holiday': 'drop',
 'working_day': 'passthrough',
 'state_of_alarm': 'passthrough'}

In [9]:
meteo_values[0]

{'rain': 'passthrough',
 'wind': 'drop',
 'temperature': 'passthrough',
 'humidity': 'passthrough',
 'pressure': 'drop',
 'radiation': 'passthrough'}

In [10]:
training_time

{(0, 18000): 0.031229496002197266,
 (0, 18001): 0.03124070167541504,
 (0, 18002): 0.031250715255737305,
 (0, 18003): 0.022092342376708984,
 (0, 18004): 0.015621662139892578,
 (0, 18005): 0.031235933303833008,
 (0, 18006): 0.03126406669616699,
 (0, 18007): 0.02189946174621582,
 (0, 18008): 0.031237363815307617,
 (0, 18009): 0.031244993209838867,
 (0, 18010): 0.03119659423828125,
 (0, 18011): 0.022088289260864258,
 (0, 18012): 0.031236648559570312,
 (0, 18013): 0.03121352195739746,
 (0, 18014): 0.022165536880493164,
 (0, 18015): 0.03123950958251953,
 (0, 18016): 0.031205177307128906,
 (0, 18017): 0.031250953674316406,
 (0, 18018): 0.02212238311767578,
 (0, 18019): 0.015571117401123047,
 (0, 18020): 0.031213045120239258,
 (0, 18021): 0.015627622604370117,
 (0, 18022): 0.015621423721313477,
 (0, 18023): 0.031201601028442383,
 (0, 18024): 0.03125190734863281,
 (0, 18025): 0.02212834358215332,
 (0, 18026): 0.031212568283081055,
 (0, 18027): 0.032819509506225586,
 (0, 18028): 0.03621268272399

In [11]:
mean_train_time = sum(list(training_time.values())) / len(list(training_time.values()))
mean_train_time

0.03782681574424108

In [12]:
alphas

{(0, 18000): [10.0, 10.0, 10.0],
 (0, 18001): [10.0, 10.0, 10.0],
 (0, 18002): [10.0, 10.0, 10.0],
 (0, 18003): [10.0, 10.0, 10.0],
 (0, 18004): [10.0, 10.0, 10.0],
 (0, 18005): [10.0, 10.0, 10.0],
 (0, 18006): [10.0, 10.0, 10.0],
 (0, 18007): [10.0, 10.0, 10.0],
 (0, 18008): [10.0, 10.0, 10.0],
 (0, 18009): [10.0, 10.0, 10.0],
 (0, 18010): [10.0, 10.0, 10.0],
 (0, 18011): [10.0, 10.0, 10.0],
 (0, 18012): [10.0, 10.0, 10.0],
 (0, 18013): [10.0, 10.0, 10.0],
 (0, 18014): [10.0, 10.0, 10.0],
 (0, 18015): [10.0, 10.0, 10.0],
 (0, 18016): [10.0, 10.0, 10.0],
 (0, 18017): [10.0, 10.0, 10.0],
 (0, 18018): [10.0, 10.0, 10.0],
 (0, 18019): [10.0, 10.0, 10.0],
 (0, 18020): [10.0, 10.0, 10.0],
 (0, 18021): [10.0, 10.0, 10.0],
 (0, 18022): [10.0, 10.0, 10.0],
 (0, 18023): [10.0, 10.0, 10.0],
 (0, 18024): [10.0, 10.0, 10.0],
 (0, 18025): [10.0, 10.0, 10.0],
 (0, 18026): [10.0, 10.0, 10.0],
 (0, 18027): [10.0, 10.0, 10.0],
 (0, 18028): [10.0, 10.0, 10.0],
 (0, 18029): [10.0, 10.0, 10.0],
 (0, 18030